In [1]:
from cycada_utils import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tr = CyDataset()
train, val = train_test_split(tr, test_size=0.2, random_state=42)
train_dataloader = DataLoader(
    train,
    batch_size=5,
    shuffle=True)

val_dataloader = DataLoader(
    val,
    batch_size=1,
    shuffle=True)

Figure directory exists.


In [99]:
#cant use softmax when i have a single output.
class FeatureDiscriminator(nn.Module):
    def __init__(self):
        super(FeatureDiscriminator, self).__init__()
        self.discriminator = nn.Sequential(
            nn.Linear(1, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 1),
            nn.Sigmoid()
        )

    def forward(self, score):
        out = self.discriminator(score)
        return out
mod = FeatureDiscriminator()
x = mod(ft)
x

tensor([[0.5081],
        [0.5117],
        [0.5054],
        [0.5125],
        [0.5147]], grad_fn=<SigmoidBackward0>)

In [78]:
mod = NLayerDiscriminator(3, 64, 4, nn.InstanceNorm2d)
img = torch.rand((5, 3,224,224))
ft=torch.rand((5,1))
yy = mod(img)
target_label = torch.zeros(x.size()[0]).long()
# x.shape

In [107]:
def process_discriminator_output(output_tensor):
    # Ensure binary_tensor is on the same device as output_tensor
    binary_tensor = torch.where(output_tensor > 0, torch.tensor(1).to(output_tensor.device), torch.tensor(0).to(output_tensor.device))
    
    # Count the number of 1s in each subarray
    count_ones = torch.sum(binary_tensor, dim=(2, 3))
#     print(count_ones)
    # Ensure majority_tensor is on the same device as output_tensor
    majority_tensor = torch.where(count_ones > 72, torch.tensor(1).to(output_tensor.device), torch.tensor(0).to(output_tensor.device))
    
    return majority_tensor.view(-1)



In [80]:
yy = process_discriminator_output(yy)
yy.shape

tensor([[80],
        [84],
        [81],
        [82],
        [72]])


torch.Size([5])

In [81]:
yy

tensor([1, 1, 1, 1, 0])

In [102]:
def prediction(out, target=None, feature=False):
    device = out.device  # Get the device of the input tensor
    
#     if target is None:
#         _, label = torch.max(out.data, 1)
#         return label.to(device)
#     else:
    if feature:
        threshold = torch.tensor([0.5], device=device)
        label = torch.where(out.data > threshold, 
                            torch.tensor(1).to(device), 
                            torch.tensor(0).to(device)).squeeze(1)
    else: label = out

    acc = (label == target).sum().item() / target.size()[0]
    return acc

In [83]:
prediction(yy, target_label,feature=False)

tensor([1, 1, 1, 1, 0])
tensor([1, 1, 1, 1, 0])
tensor([0, 0, 0, 0, 0])
0.2


0.2

In [97]:
x

tensor([[0.5016],
        [0.4994],
        [0.5023],
        [0.4985],
        [0.4953]], grad_fn=<SigmoidBackward0>)

In [98]:
prediction(x, target=target_label, feature=True)

0.6

In [110]:
class Feature_Level_Adaptation():
    def __init__(self, gan_mode='wgangp'):
        super(Feature_Level_Adaptation,self).__init__()
        self.gan_mode = gan_mode
        
        generator_checkpoint = torch.load('/datacommons/carlsonlab/srs108/cicada/pixel_level/best_gens.pt')
        discriminator_checkpoint1 = torch.load('/datacommons/carlsonlab/srs108/cicada/pixel_level/best_DS.pt')
        discriminator_checkpoint2 = torch.load('/datacommons/carlsonlab/srs108/cicada/pixel_level/best_DT.pt')

        #PHASE ONE~source task model        
        self.f_s = Multi_City_CNN()
        self.f_s.load_state_dict(torch.load('dlm4.pt'))
        
        #PHASE TWO~pixel level adaptation
        norm_layer_G = functools.partial(nn.BatchNorm2d, affine=True, track_running_stats=True)
        norm_layer_D = functools.partial(nn.InstanceNorm2d, affine=False)
        self.G_ST = ResnetGenerator(3, 3, 64, norm_layer=norm_layer_G, use_dropout=False, n_blocks=2)
        self.G_ST.load_state_dict(generator_checkpoint['G_ST'])
        
        self.G_TS = ResnetGenerator(3, 3, 64, norm_layer=norm_layer_G, use_dropout=False, n_blocks=2)
        self.G_TS.load_state_dict(generator_checkpoint['G_TS'])
        
        self.D_T = NLayerDiscriminator(input_nc=3, ndf=64, n_layers=4, norm_layer=norm_layer_D)
        self.D_T.load_state_dict(discriminator_checkpoint2['D_T'])
        self.D_S = NLayerDiscriminator(input_nc=3, ndf=64, n_layers=4, norm_layer=norm_layer_D)
        self.D_S.load_state_dict(discriminator_checkpoint1['D_S'])
        
        #PHASE THREE~feature space adaptation
        self.D_ft = FeatureDiscriminator()
        self.f_t = LeNet(3)

        self.G_ST.to(device)
        self.G_TS.to(device)
        self.D_S.to(device)
        self.D_T.to(device)
        self.f_s.to(device)
        self.D_ft.to(device)
        self.f_t.to(device)


        self.ganloss = GANLoss(gan_mode=self.gan_mode).to(device)       #use to fool discriminator
        self.cycleloss = torch.nn.L1Loss().to(device)               #difference between reconstructed img and original
        self.mseloss = torch.nn.MSELoss().to(device)       #difference between domain classifications between input img and generator output
        self.identityloss = torch.nn.L1Loss().to(device)
        self.ftloss = torch.nn.modules.CrossEntropyLoss().to(device)            #difference between generator output from input img and input img

        
        self.optimizer_G = torch.optim.Adam(itertools.chain(self.G_ST.parameters(), self.G_TS.parameters()), lr=2e-5, betas=(0.5,0.999))
        self.optimizer_DS = torch.optim.Adam(self.D_S.parameters(), lr = 1e-5, betas = (0.5,0.9))
        self.optimizer_DT = torch.optim.Adam(self.D_T.parameters(), lr = 1e-5, betas = (0.5,0.9))
        
        self.optimizer_f_s = torch.optim.Adam(self.f_s.parameters(), lr=1e-4, betas=(0.5, 0.999))    
        self.optimizer_f_t = torch.optim.Adam(self.f_t.parameters(), lr=1e-5, betas=(0.5, 0.999))    #different learning rate??
        self.optimizer_D_ft = torch.optim.Adam(self.D_ft.parameters(), lr = 1e-5, betas = (0.5,0.9))

        
        self.D_ft.apply(weights_init_normal)
        self.f_t.apply(weights_init_normal)

        
    def data_input(self, batch):
        self.real_S = batch['D img'].type(Tensor)
        self.real_T = batch['L img'].type(Tensor)
        self.real_lbl = batch['D pm'].type(Tensor).float()

        self.source_label = torch.ones(self.real_S.size()[0]).long().to(device)
        self.target_label = torch.zeros(self.real_T.size()[0]).long().to(device)
        
    def sample_images(self, dataloader, epochs, iters, save = False):
        source = next(iter(dataloader))
        self.G_ST.eval()
        self.G_TS.eval()
        real_source = source['D img'].type(Tensor) 
        fake_target = self.G_ST(real_source).detach()
        real_target = source['L img'].type(Tensor)
        real_lbl= source['D pm'].type(Tensor).float()
        fake_source = self.G_TS(real_target).detach()

        recons = self.G_TS(fake_target).detach()
        recont = self.G_ST(fake_source).detach() 

        real_S = make_grid(real_source, nrow=5, normalize=True, scale_each=True, padding=1)
        fake_T = make_grid(fake_target, nrow=5, normalize=True, scale_each=True, padding=1)
        reconS = make_grid(recons, nrow=5, normalize=True, scale_each=True, padding=1)
        real_T = make_grid(real_target, nrow=5, normalize=True, scale_each=True, padding=1)
        fake_S = make_grid(fake_source, nrow=5, normalize=True, scale_each=True, padding=1)
        reconT = make_grid(recont, nrow=5, normalize=True, scale_each=True, padding=1)
        # Arange images along y-axis    
        image_grid = torch.cat((real_S, fake_T, reconS, real_T, fake_S, reconT), 2)
        plt.imshow(image_grid.cpu().permute(1,2,0))
        plt.title('Real Source vs Fake Target vs Recon Source | Real Target vs Fake Source vs Recon Target')
        plt.axis('off')
        plt.gcf().set_size_inches(10, 6)
        if save:
            plt.savefig(os.path.join('Figure_PDFs', f'epoch_{str(e+1)}_iter{str(i+1)}.png'), bbox_inches='tight', pad_inches=0, facecolor='white')
        plt.show();
        
    def forward_pass(self):
        self.fake_t = self.G_ST(self.real_S)
        self.fake_s = self.G_TS(self.real_T)             
        
        self.recov_s = self.G_TS(self.fake_t)
        self.recov_t = self.G_ST(self.fake_s)
        
        #compute MSE from PM values ***predicted*** from real image and fake image (same pm vals tho!!)
        #DO NOT ASSESS WITH REAL PM LABEL, BUT WITH PREDICTION
        
        self.pm_from_real_S  = self.f_s(self.real_S).squeeze(1)
        self.pm_from_fake_T  = self.f_s(self.fake_t.detach()).squeeze(1)
        
        self.pm_from_real_T  = self.f_s(self.real_T).squeeze(1)
        self.pm_from_fake_S  = self.f_s(self.fake_s.detach()).squeeze(1)
        
        self.pred_source  = self.D_ft(self.f_t(self.fake_t.detach()))
        self.pred_target  = self.D_ft(self.f_t(self.real_T))

        self.dft_acc_1 = prediction(self.pred_source,self.source_label, feature = True)
        self.dft_acc_2 = prediction(self.pred_target, self.target_label, feature = True)

        self.score_acc_D_ft = (self.dft_acc_1 + self.dft_acc_2)/2
        print(self.score_acc_D_ft)
    def source_task(self):
        #L_task(f_s, G_ST(S), p(fS,S))
        self.sem_source  = self.mseloss(self.pm_from_real_S, self.pm_from_fake_T)  #MSE(f_s(S), f_s(G_ST(S)))                  

        #L_task(f_s, G_TS(T), p(fS,T))
        self.sem_targ    = self.mseloss(self.pm_from_real_T, self.pm_from_fake_S)  #MSE(f_s(T), f_s(G_TS(T))) 
        self.sem_loss  = (self.sem_source + self.sem_targ)/2
        
        self.optimizer_f_s.step
        
    def backward_D(self, netD, real, noise):
         
        # Real
        pred_real = netD(real)
        if self.gan_mode == 'wgangp':
            loss_D_real = -torch.mean(pred_real)
        else: loss_D_real = self.ganloss(pred_real, True)
        
        # Fake
        pred_fake = netD(noise.detach())
        if self.gan_mode == 'wgangp':
            loss_D_fake = torch.mean(pred_fake)
        else: loss_D_fake = self.ganloss(pred_fake, False)
        
        # Combined loss and calculate gradients

        if self.gan_mode == "wgangp":
            gradient_penalty, gradients = cal_gradient_penalty(netD,real,noise,device)
            gradient_penalty.backward(retain_graph=True)
        
            loss_D = loss_D_real + loss_D_fake + 10 * ((gradients.norm(2, dim=1) - 1) ** 2).mean()

        else:
            loss_D = (loss_D_real + loss_D_fake) * 0.5

        true_labels = torch.ones(real.size()[0]).long()
        fake_labels = torch.zeros(noise.detach().size()[0]).long()
        pred_real   = process_discriminator_output(pred_real)
        pred_fake   = process_discriminator_output(pred_fake)
        true_acc = prediction(pred_real.cpu(), true_labels,feature=False)
        fake_acc = prediction(pred_fake.cpu(), fake_labels,feature=False)
        acc = (true_acc + fake_acc) * 0.5
        return loss_D , acc

        
    def backward_DS(self):
        self.DS_Loss_wgan, _ = self.backward_D(self.D_S, self.real_S ,self.fake_s.detach())
#         self.DS_Loss_wgan.backward()
        self.optimizer_DS.step()
        
    def backward_DT(self):
        self.DT_Loss_wgan, _ = self.backward_D(self.D_T, self.real_T, self.fake_t.detach())
#         self.DT_Loss_wgan.backward()
        self.optimizer_DT.step()
        
    def backward_D_feat(self):
        
        self.pred_source  = self.D_ft(self.f_t(self.fake_t.detach())).squeeze(1)
        self.pred_target  = self.D_ft(self.f_t(self.real_T)).squeeze(1)
        
        loss_D_ft_s = self.ganloss(self.pred_source, Fake)
#         loss_D_ft_s = self.ftloss(self.pred_source.type(Tensor), self.source_label.type(Tensor))
       
        # Target
        loss_D_ft_t = self.ganloss(self.pred_target, True)
#         loss_D_ft_t = self.ftloss(self.pred_target.type(Tensor), self.target_label.type(Tensor))
        # Combined loss
        
        self.loss_D_ft_adv = (loss_D_ft_s + loss_D_ft_t) * 0.5
        self.loss_D_ft_adv.backward()
        self.optimizer_D_ft.step()

        
    def backward_G(self):
        #GAN loss (identity loss in cycleGAN/pixel level for cycada)
        
        self.idt_S = self.G_ST(self.real_T)  #G_ST(G_TS(t)) ~ s
        self. idt_T = self.G_TS(self.real_S)  #G_TS(G_ST(s))
        self.loss_idt_S = self.identityloss(self.idt_S, self.real_T) * 10
        self.loss_idt_T = self.identityloss(self.idt_T, self.real_S) * 10
        
        if self.gan_mode == 'wgangp':
            self.DS_GST_S = self.D_S(self.fake_t)
            self.loss_G_S = -torch.mean(self.DS_GST_S)

            self.DT_GTS_T = self.D_T(self.fake_s)
            self.loss_G_T = -torch.mean(self.DT_GTS_T)
            
        else:
            self.loss_G_S = self.ganloss(self.D_S(self.fake_t), True)       #L(D_S(G_ST(S)))
            self.loss_G_T = self.ganloss(self.D_T(self.fake_s), True)       #L(D_T(G_TS(T)))    
        
        # cycle loss  (L1 Loss)      
        self.loss_cycle_S   = self.cycleloss(self.recov_s, self.real_S) * 10   # Lcyc(G_TS(G_ST(S)), S) * λ 
        self.loss_cycle_T   = self.cycleloss(self.recov_t, self.real_T) * 10  # Lcyc(G_ST(G_TS(T)), T) * λ 
        
        self.loss_G = self.loss_G_S + self.loss_G_T + self.loss_cycle_S + self.loss_cycle_T + self.loss_idt_S + self.loss_idt_T

        self.sem_loss_source    = self.mseloss(self.pm_from_real_S, self.pm_from_fake_T)  #Ltask(fs, G_TS(T), p(fs, T))                   
        self.sem_loss_targ      = self.mseloss(self.pm_from_real_T,self.pm_from_fake_S)   #Ltask(fs, G_ST(S), p(fs, S))
        
        self.loss_G += self.sem_loss_source + self.sem_loss_targ
#         self.loss_G.backward()
        self.optimizer_G.step()
    
    def backward_ft(self):
        main_task_pred = self.f_t(self.fake_t.detach()).squeeze(1)
        main_task_loss = self.mseloss(main_task_pred, self.real_lbl)
        
        if self.score_acc_D_ft > 0.6:
            pred_target_main = self.D_ft(self.f_t(self._T))  #Dft(ft(T)) 
            
        main_task_loss.backward()
        self.optimizer_f_t.step()
        
    def optimize(self):
        self.forward_pass()
        
#         fs (pretrained already, no backprop)
        set_requires_grad([self.f_s],requires_grad=False)
        self.optimizer_f_s.zero_grad()
        self.source_task()
        
# -------------------------------------------------------------------------------------------------------
# Generators G_ST and G_TS ;
# -------------------------------------------------------------------------------------------------------
        set_requires_grad([self.D_S, self.D_T],requires_grad=False)
        set_requires_grad([self.G_ST, self.G_TS],requires_grad=False)
        self.optimizer_G.zero_grad()
        self.optimizer_f_s.zero_grad()
        self.backward_G()
        
# -------------------------------------------------------------------------------------------------------
#  Discriminators S and Critic T;
# -------------------------------------------------------------------------------------------------------
  
#         for i in range(5):
        self.optimizer_DS.zero_grad()
        self.backward_DS()
        
#         for _ in range(5):
        self.optimizer_DT.zero_grad()
        self.backward_DT()
        
        
# -------------------------------------------------------------------------------------------------------
#  Phase 3; Feature Discriminator and task model
# -------------------------------------------------------------------------------------------------------

        set_requires_grad([self.f_t], True)
        set_requires_grad([self.D_ft], False)
        
        self.optimizer_f_t.zero_grad()
        self.backward_ft()
        
        
        set_requires_grad([self.f_t], False)
        set_requires_grad([self.D_ft], True)
        self.optimizer_D_ft.zero_grad()
        self.backward_D_feat()
        return self.loss_G, self.DS_Loss_wgan, self.DT_Loss_wgan, self.D_S, self.D_T, self.G_ST, self.G_TS

In [111]:
history = {'epoch':[],'G_loss': [], 'DS_loss':[], 'DT_loss':[], 'batch':[]}
torch.cuda.empty_cache()
model = Feature_Level_Adaptation(gan_mode='wgangp')
best_gen_loss = 1e6
best_DT_loss = 1e6
best_DS_loss = 1e6

n_epochs = 200

for e in range(n_epochs):
    for i, batch in tqdm(enumerate(train_dataloader)):
        model.data_input(batch)
        G_loss, DS_loss, DT_loss, DS, DT, GST, GTS = model.optimize()
#         model.sample_images(val_dataloader, e, i, save=True)
#         if DS_loss < best_DS_loss:
#             best_DS_loss = DS_loss
#             torch.save({ 'D_S': DS.state_dict()}, 'best_DS.pt')
 
#         if DT_loss < best_DT_loss:
#             best_DT_loss = DT_loss
#             torch.save({'D_T': DT.state_dict()}, 'best_DT.pt')
            
#         if G_loss < best_gen_loss:
#             best_gen_loss = G_loss
#             torch.save({ 'G_ST': GST.state_dict(),'G_TS': GTS.state_dict()}, 'best_gens.pt')
            
        
#         if (i+1) % 250 == 0:
#             with torch.no_grad():
#                 model.sample_images(val_dataloader, e, i, save=True)
                
#                 history['G_loss'].append(G_loss.item())
#                 history['DS_loss'].append(DS_loss.item())
#                 history['DT_loss'].append(DT_loss.item())
#                 history['batch'].append(i+1)
#                 history['epoch'].append(e+1)
#     now = datetime.datetime.now()
#     print(f"Epoch {e + 1}/{n_epochs} finished at {now.time()}\n\
#     [G Loss: {G_loss.item()}]\t[D Loss: {DS_loss.item()+ DT_loss.item()}]")

0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.7
0.8
0.6
0.7
0.5
0.5
0.5
0.5
0.2
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.4
0.4
0.5
0.5
0.4
0.5
0.5
0.5
0.4
0.5
0.5
0.5
0.4
0.5
0.5
0.5
0.4
0.3
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.4
0.5
0.5
0.5



KeyboardInterrupt: 